# Grid sampling

In [50]:
%cd "~/Repositories/NB10422645"
from src.problems.two_dimensional import *
import pandas as pd
test_geometry = MissionGeometry()
test_design = NOmegaPointsScaleBasedPeriodic(int(test_geometry.P_s / 10), 3)
test_spacecraft = Spacecraft()
test_problem = AttitudeTrajectoryProblem2D(test_geometry, test_spacecraft, test_design)

/home/ggarrett/Repositories/NB10422645


## Unbiased Domain Grid Sampling

In [10]:
def grid_sample_domain(bounds, n_sample_dim, flatten=True):
    linspace = [np.linspace(bounds[0][i], bounds[1][i], n_sample_dim) for i in range(len(bounds[0]))]
    _return = np.array(np.meshgrid(*linspace))
    return np.array([_return_i.flatten() for _return_i in _return.reshape(len(bounds[0]), -1)]).T if flatten else _return

In [36]:
%%timeit
res1 = grid_sample_domain([[0,0],[1,1]], 3)
np.round(res1, 2)

115 µs ± 1.58 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


## Random Domain Sampling

In [13]:
def random_uniform_sample_domain(bounds, n_samples, seed=101):
    np.random.seed(seed)
    return np.random.uniform(bounds[0], bounds[1], (n_samples, len(bounds[0])))

In [37]:
%%timeit
res2 = random_uniform_sample_domain([[0,0],[1,1]], 3**2)
np.round(res2, 2)

21.6 µs ± 734 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


## Halton Domain Sampling

In [31]:
"""Halton low discrepancy sequence.
This snippet implements the Halton sequence following the generalization of
a sequence of *Van der Corput* in n-dimensions.
---------------------------
MIT License
Copyright (c) 2017 Pamphile Tupui ROY
Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:
The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.
THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.
"""
import numpy as np


def primes_from_2_to(n):
    """Prime number from 2 to n.
    From `StackOverflow <https://stackoverflow.com/questions/2068372>`_.
    :param int n: sup bound with ``n >= 6``.
    :return: primes in 2 <= p < n.
    :rtype: list
    """
    sieve = np.ones(n // 3 + (n % 6 == 2), dtype=np.bool)
    for i in range(1, int(n ** 0.5) // 3 + 1):
        if sieve[i]:
            k = 3 * i + 1 | 1
            sieve[k * k // 3::2 * k] = False
            sieve[k * (k - 2 * (i & 1) + 4) // 3::2 * k] = False
    return np.r_[2, 3, ((3 * np.nonzero(sieve)[0][1:] + 1) | 1)]


def van_der_corput(n_sample, base=2):
    """Van der Corput sequence.
    :param int n_sample: number of element of the sequence.
    :param int base: base of the sequence.
    :return: sequence of Van der Corput.
    :rtype: list (n_samples,)
    """
    sequence = []
    for i in range(n_sample):
        n_th_number, denom = 0., 1.
        while i > 0:
            i, remainder = divmod(i, base)
            denom *= base
            n_th_number += remainder / denom
        sequence.append(n_th_number)

    return sequence


def halton(dim, n_sample):
    """Halton sequence.
    :param int dim: dimension
    :param int n_sample: number of samples.
    :return: sequence of Halton.
    :rtype: array_like (n_samples, n_features)
    """
    big_number = 10
    while 'Not enought primes':
        base = primes_from_2_to(big_number)[:dim]
        if len(base) == dim:
            break
        big_number += 1000

    # Generate a sample using a Van der Corput sequence per dimension.
    sample = [van_der_corput(n_sample + 1, dim) for dim in base]
    sample = np.stack(sample, axis=-1)[1:]

    return sample

def halton_sample_domain(bounds, n_samples):
    dim = len(bounds[0])
    bounds = np.array(bounds)
    halton_sample = halton(dim, n_samples)
    return halton_sample * (bounds[1,:]-bounds[0,:]) + bounds[0,:]

In [51]:
%%timeit
res3 = halton_sample_domain([[0,0],[1,1]], 3**2)
np.round(res3,2)

264 µs ± 9.94 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [52]:
def process_samples(samples, problem):
    column_dim = len(problem.get_bounds()[0]) + problem.fitness_dim
    res=np.zeros((samples.shape[0], column_dim))
    count=0
    perc_val = int(0.1 * samples.shape[0])
    for idx,_x in enumerate(samples):
        count+=1
        f = problem.fitness(_x)
        res[idx] = np.concatenate((f,_x))
        if (count % perc_val) == 0:
            print(round(count/len(samples) * 100, 1), r"%")
    return res

In [ ]:
# Save random dataset.

sample_random_data = random_uniform_sample_domain(test_problem.get_bounds(), 6**7)
result_random_data = process_samples(sample_random_data, test_problem)
df_random_sampling = pd.DataFrame(columns=[f"f{i+1}" for i in range(test_problem.fitness_dim)] + [f"x{i+1}" for i in range(len(test_problem.design_space.get_bounds()[0]))], data=result_random_data)
df_random_sampling.to_parquet(f"data/dataSet2_randomSampled_6.parquet")

In [ ]:
# Save halton dataset.

sample_halton_data = halton_sample_domain(test_problem.get_bounds(), 6**7)
result_halton_data = process_samples(sample_halton_data, test_problem)
df_halton_sampling = pd.DataFrame(columns=[f"f{i+1}" for i in range(test_problem.fitness_dim)] + [f"x{i+1}" for i in range(len(test_problem.design_space.get_bounds()[0]))], data=result_halton_data)
df_halton_sampling.to_parquet(f"data/dataSet2_haltonSampled_6.parquet")

In [ ]:

def grid_sample_problem(problem, n=10, scale_bounds=1.0, fixed=(())):
    
    bounds = scale_bounds * np.array(problem.design_space.get_bounds())    
    grid = sampled_grid(bounds, n, flatten=True)
    grid = pd.DataFrame(grid.T, columns=[f"x{i+1}" for i in range(len(grid.T[0]))])
    grid_ = grid
    grid_values = grid_.values
    perc_val = int(0.1 * len(grid_values))
    
    # Generate storage matrix.
    res=np.zeros((grid_.values.shape[0], 12))   # Add f,x shape attribute to problem.
    count=0
    for idx,_x in enumerate(grid_values):
        count+=1
        f = problem.fitness(_x)
        res[idx] = np.concatenate((f,_x))
        if (count % perc_val) == 0:
            print(round(count/len(grid_values) * 100, 1), r"%")
        
    df = pd.DataFrame(columns=[f"f{i+1}" for i in range(problem.fitness_dim)] + [f"x{i+1}" for i in range(len(test_problem.design_space.get_bounds()[0]))], data=res)
#     if os.path.exists("data"):
#         pass
#     else:
#         os.mkdir("data")
        
    save_name = f"data/AttitudeTrajectoryProblem2D_C1_R1/dataSet2_gridSampled_{n}.parquet"
    
    df.to_parquet(save_name)


In [ ]:
for n in np.arange(8,12):
    grid_sample_problem(test_problem, n)
    

## Case 2: Grid Sampled Optimal Region

In [6]:
def grid_sample_solution(problem, solution, n, scale_per_bound=0.025):
#     print(x)
    
    # Get problem bounds.
    bounds = np.array(problem.design_space.get_bounds())
    
    # Transform into bound range.
    bound_range = bounds[1]-bounds[0]
    
#     print(bound_range)
    
    # Scale bound range for about point basis.
    scaled_bound_range = scale_per_bound * bound_range

    # Create new bounds centralised about point.
    bounds = np.concatenate(([solution+scaled_bound_range],[solution-scaled_bound_range]))
    
#     print(bounds.shape)
    
    # Sample bounds about point.
    grid = sampled_grid(bounds, n, flatten=True)
    
#     print(grid.shape)
    
    # Transform to dataframe.
    grid = pd.DataFrame(grid.T, columns=[f"x{i+1}" for i in range(len(grid.T[0]))])
    grid_ = grid
    grid_values = grid_.values
    perc_val = int(0.1 * len(grid_values))
    
    # Generate storage matrix.
    res=np.zeros((grid_.values.shape[0], 12))   # Add f,x shape attribute to problem.
    count=0
    for idx,_x in enumerate(grid_values):
        count+=1
#         print(_x)
        f = problem.fitness(_x)
        res[idx] = np.concatenate((f,_x))
        if (count % perc_val) == 0:
            print(round(count/len(grid_values) * 100, 1), r"%")
        
    df = pd.DataFrame(columns=[f"f{i+1}" for i in range(problem.fitness_dim)] + [f"x{i+1}" for i in range(len(test_problem.design_space.get_bounds()[0]))], data=res)
        
    save_name = f"data/AttitudeTrajectoryProblem2D_C1_R1/dataSet2_sol1_gridSampled_{n}.parquet"
    
    df.to_parquet(save_name)


x = np.array([0.00192016,
             -0.00135844,
              0.00226742,
              0.4064695,
              0.28561942,
              0.96828936,
              0.50069615])

x[-4:] = x[-4:] / np.linalg.norm(x[-4:])

In [ ]:
for n in np.arange(3,12):
    grid_sample_solution(test_problem, x, n, scale_per_bound=0.025)
    

